In [ ]:
import torch 
import torch_geometric.transforms
import torch_geometric.data
from torch_geometric.data import Data, ClusterData, DataLoader
import numpy as np
import pickle
from torch.utils.data import random_split
import random
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphConv, TopKPooling,GATConv,SAGPooling,LEConv,ASAPooling
from torch_geometric.data import Batch
from torch_scatter import scatter_add
from torch.nn import Conv1d


In [ ]:
#dataset = torch.load('graphs_stdc_dmixed.pt')

dataset = torch.load('../data/stdc/d7/graphs_clean.pt')
#dataset = torch.load('../data/stdc/dmixed_large/graphs_clean.pt')
#dataset = torch.load('../data/stdc/dmixed(579)_p1/graphs_clean.pt')
#dataset = torch.load('../data/stdc/d5p1/graphs_clean.pt')


batchsize=128
batch_train_splits = int(0.8*len(dataset)/batchsize)*batchsize
#batch_val_splits = int(0.2*len(dataset)/batchsize)*batchsize
batch_test_splits = int(((len(dataset)-batch_train_splits)/batchsize)*batchsize)
#pint(batch_train_splits)
#print(batch_test_splits)

dataset_usage = len(dataset)-(batch_train_splits+batch_test_splits)

if dataset_usage ==0:
    train_dataset, test_dataset = random_split(dataset[:],[batch_train_splits, batch_test_splits],generator=torch.Generator().manual_seed(42))

else:
    train_dataset, test_dataset = random_split(dataset[:-dataset_usage],[batch_train_splits,batch_test_splits],generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)
#val_loader = DataLoader(val_dataset, batch_size=batchsize, shuffle=False)

print(len(test_loader))
print(len(train_loader))
#print(len(val_loader))

In [ ]:
class GraphConv_net(torch.nn.Module):
    def __init__(self,hidden_channels,k,device):
        super(GraphConv_net, self).__init__()
        
        self.conv = GraphConv(2,hidden_channels,aggr='add')
        self.conv_1 = GraphConv(hidden_channels,hidden_channels,aggr='add')
        
        #self.conv = LEConv(2,hidden_channels)
        #self.conv_1 = LEConv(hidden_channels,hidden_channels)
        
        #self.topkpoolX_test = SAGPooling(hidden_channels,ratio=int(k),GNN=GATConv)
        #self.topkpoolZ_test = SAGPooling(hidden_channels,ratio=int(k),GNN=GATConv)

        #self.topkpoolX = SAGPooling(hidden_channels,ratio=int(k))
        #self.topkpoolZ = SAGPooling(hidden_channels,ratio=int(k))

        
        
        self.topkpoolX_test = TopKPooling(hidden_channels,ratio=int(k))
        self.topkpoolZ_test = TopKPooling(hidden_channels,ratio=int(k))

        self.topkpoolX = TopKPooling(hidden_channels,ratio=int(k))
        self.topkpoolZ = TopKPooling(hidden_channels,ratio=int(k))

        
        self.lin = torch.nn.Linear(2*(hidden_channels*k),48)
        self.lin2 = torch.nn.Linear(48,12)
        
        self.lin_f = torch.nn.Linear(12,1)
        self.tanh = torch.nn.Tanh()
        self.lrelu = torch.nn.LeakyReLU(negative_slope=0.2)
        self.sigmoid = torch.nn.Sigmoid()
        self.drop_out = torch.nn.Dropout(p=0.2)
        
        self.hidden_channels = hidden_channels
        self.k = k
        
    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        x = self.conv(x,edge_index,edge_weight=edge_weight)
        x = self.tanh(x)
        #x = self.conv1(x,edge_index,edge_weight=edge_weight)

        x_test, edge_index_xtest, edge_attr_xtest, batch_xtest ,_,_ = self.topkpoolX_test(x,edge_index, batch=batch,edge_attr=edge_weight)
        z_test, edge_index_ztest, edge_attr_ztest ,batch_ztest, _,_= self.topkpoolZ_test(x,edge_index, edge_attr=edge_weight, batch=batch)

        x = self.conv_1(x, edge_index, edge_weight=edge_weight)
        
        
        X, edge_index_1, edge_attr_1, batch_1 ,_,_ = self.topkpoolX(x,edge_index, edge_attr=edge_weight, batch=batch)
        Z, edge_index_2, edge_attr_2 ,batch_2, _,_ = self.topkpoolZ(x,edge_index, edge_attr=edge_weight, batch=batch)


        
        #print(x_test.shape)
        
        if x_test.size(0)<(batch_xtest[-1]+1)*self.k:
            #missing_dim = (batch_1[-1]+1)*self.k-X.size(0)

            old_length_x = 0
            for jx in range(batch_xtest[-1]+1):
                if jx == batch_xtest[-1]:
                    length_x = len(batch_xtest[batch_xtest==jx])
                    first_part = x_test[0:length_x+old_length_x, :]
                    new_row = torch.zeros(self.k-length_x, self.hidden_channels,device=device)#*self.num_heads
                    x_test = torch.cat([first_part, new_row],dim=0)
                else:
                    length_x = len(batch_xtest[batch_xtest==jx])
                    first_half = x_test[0:length_x+old_length_x, :]
                    second_half = x_test[length_x+old_length_x:, :]
                    x_add = torch.zeros(self.k-length_x, self.hidden_channels,device=device)#*self.num_heads
                    x_test = torch.cat([first_half, x_add, second_half],dim=0)
                    
                    
                    old_length_x += self.k

        if z_test.size(0)<(batch_ztest[-1]+1)*self.k:

            old_length_z = 0
            for kx in range(batch_ztest[-1]+1):
                #if Z.size(0) == (batch_2[-1]+1)*self.k:
                #    continue
                if kx == batch_ztest[-1]:
                    length_z = len(batch_ztest[batch_ztest==kx])
                    first_part = z_test[0:length_z+old_length_z, :]
                    new_row = torch.zeros(self.k-length_z, self.hidden_channels,device=device)#self.num_heads
                    z_test = torch.cat([first_part, new_row],dim=0)
                else:
                    length_z = len(batch_ztest[batch_ztest==kx])
                    first_half = z_test[0:length_z+old_length_z, :]
                    second_half = z_test[length_z+old_length_z:, :]
                    z_add = torch.zeros(self.k-length_z, self.hidden_channels,device=device)#self.num_heads
                    z_test = torch.cat([first_half, z_add, second_half],dim=0)
                    old_length_z += self.k
                    
        
        
        
        x_test = torch.flatten(x_test)
        z_test = torch.flatten(z_test)
        

        x_test = x_test.view(batch_xtest[-1]+1,self.hidden_channels*self.k)#self.num_heads*2)
        z_test = z_test.view(batch_ztest[-1]+1,self.hidden_channels*self.k)#self.num_heads*2)
                
        
        if X.size(0)<(batch_1[-1]+1)*int(self.k):
            #missing_dim = (batch_1[-1]+1)*self.k-X.size(0)

            old_length_x = 0
            for jx in range(batch_1[-1]+1):
                if jx == batch_1[-1]:
                    length_x = len(batch_1[batch_1==jx])
                    first_part = X[0:length_x+old_length_x, :]
                    new_row = torch.zeros(int(self.k)-length_x, self.hidden_channels,device=device)#*self.num_heads
                    X = torch.cat([first_part, new_row],dim=0)
                else:
                    length_x = len(batch_1[batch_1==jx])
                    first_half = X[0:length_x+old_length_x, :]
                    second_half = X[length_x+old_length_x:, :]
                    x_add = torch.zeros(int(self.k)-length_x, self.hidden_channels,device=device)#*self.num_heads
                    X = torch.cat([first_half, x_add, second_half],dim=0)
                    old_length_x += int(self.k)

        if Z.size(0)<(batch_2[-1]+1)*int(self.k):

            old_length_z = 0
            for kx in range(batch_2[-1]+1):
                #if Z.size(0) == (batch_2[-1]+1)*self.k:
                #    continue
                if kx == batch_2[-1]:
                    length_z = len(batch_2[batch_2==kx])
                    first_part = Z[0:length_z+old_length_z, :]
                    new_row = torch.zeros(int(self.k)-length_z, self.hidden_channels,device=device)#self.num_heads
                    Z = torch.cat([first_part, new_row],dim=0)
                else:
                    length_z = len(batch_2[batch_2==kx])
                    first_half = Z[0:length_z+old_length_z, :]
                    second_half = Z[length_z+old_length_z:, :]
                    z_add = torch.zeros(int(self.k)-length_z, self.hidden_channels,device=device)#self.num_heads
                    Z = torch.cat([first_half, z_add, second_half],dim=0)
                    old_length_z += int(self.k)
        
        
        X = torch.flatten(X)
        Z = torch.flatten(Z)
        #print(X.shape)
        #print(Z.shape)

        X = X.view(batch_1[-1]+1,self.hidden_channels*int(self.k))#self.num_heads*2)
        Z = Z.view(batch_2[-1]+1,self.hidden_channels*int(self.k))#self.num_heads*2)
        
        X = torch.cat([x_test, X], dim=-1)
        Z = torch.cat([z_test, Z], dim=-1)
        
        #print(X.shape)
        #print(Z.shape)
        
        X = self.drop_out(X)
        Z = self.drop_out(Z)
        
        
        Z = self.lin(Z)
        X = self.lin(X)
        
        
        Z = self.tanh(Z)
        X = self.tanh(X)
        
        Z = self.lin2(Z)
        X = self.lin2(X)
        
        
        Z = self.tanh(Z)
        X = self.tanh(X)
        
        
        
        Z = self.lin_f(Z)
        X = self.lin_f(X)
        

        return self.sigmoid(X),self.sigmoid(Z)

In [ ]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device =torch.device('cpu')

In [ ]:
seed =3
learning_rate = 0.0025#0.0025 #(0.005)
torch.manual_seed(seed)

model = GraphConv_net(12,30,device)
model.to(device)
model = model.double()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.4,patience=10)


#criterion = torch.nn.MSELoss().to(device)
criterion = torch.nn.BCELoss().to(device)




def train():
    model.train()
    acc = []
    loss_holder = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        
        tar, idx = torch.max(data.y,dim=1)
        int_tar = idx.cpu().numpy()
        
        #int_tar = np.array(idx)
        #print(int_tar)
        bin_tar = ((int_tar.reshape(-1,1) & (2**np.arange(2))) != 0).astype(int)
        bin_target = bin_tar[:,::-1]
        #print(bin_target)
    
        x_target = bin_target[:,0]
        z_target = bin_target[:,1]

        #rint(x_target)
        #print(z_target)

        
        x_target = torch.from_numpy(x_target.copy()).double().to(device)
        z_target = torch.from_numpy(z_target.copy()).double().to(device)

    
        out_X, out_Z = model(data)


        loss_X = criterion(out_X, x_target.unsqueeze(-1))# Compute the loss
        loss_Z = criterion(out_Z, z_target.unsqueeze(-1))
        

        loss = sum([loss_X, loss_Z])  # or loss = loss1 + loss2
        loss_holder+=loss.item()
        loss.backward()
        
        optimizer.step()# Update parameters based on gradients.
    #acc.append([loss_X.item() ,loss_Z.item()])
    loss_item = loss_holder/len(train_loader) 
    return loss_item
    
def test(loader):
    model.eval()

    correct_Z = 0
    correct_X = 0
    correct = 0
    
    for data in loader:  # Iterate in batches over the training/test dataset.
        data = data.to(device)
        
        tar, idx = torch.max(data.y,dim=1)

        #int_tar = np.array(idx)
        int_tar = idx.cpu().numpy()
        #print(int_tar)
        bin_tar = ((int_tar.reshape(-1,1) & (2**np.arange(2))) != 0).astype(int)
        bin_target = bin_tar[:,::-1]
        #print(bin_target)
        x_target = bin_target[:,0]
        z_target = bin_target[:,1]
    

        x_target = torch.from_numpy(x_target.copy()).double().to(device)
        z_target = torch.from_numpy(z_target.copy()).double().to(device)

    

        out_X, out_Z = model(data)
        #print(out_X, out_Z)
        target = torch.cat([x_target.unsqueeze(-1),z_target.unsqueeze(-1)],dim=1)
        pred = torch.cat([out_X,out_Z],dim=1)
        pred = torch.round(pred)      

        correct_X += int((torch.round(out_X)== x_target.unsqueeze(-1)).sum())
        correct_Z += int((torch.round(out_Z)== z_target.unsqueeze(-1)).sum())
        
        for ix in range(len(pred)):
            corr=0
            corr = int((pred[ix] == target[ix]).sum())  # Check against ground-truth labels.
            if corr == 2:
                correct =correct+ 1
            
            
        
        
    return correct/len(loader.dataset), correct_X / len(loader.dataset), correct_Z/len(loader.dataset) 
    


train_list = []
loss_list = []
for epoch in range(1,250):
    loss = train()
    loss_list.append(loss)
    scheduler.step(loss)
    train_acc,train_acc_X,train_acc_Z = test(train_loader)
    test_acc ,_ ,_ = test(test_loader) 
    if epoch % 5==0:
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train Acc_X: {train_acc_X:.4f}, Train Acc_Z: {train_acc_Z:.4f}, Test Acc: {test_acc:.4f}')
    train_list.append([train_acc,test_acc,train_acc_X,train_acc_Z])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()


ax.plot(loss_list)
#ax.plot(b)

ax.set(xlabel="Epochs",ylabel="Loss")
ax.set(title ='Loss')
ax.legend()
#plt.savefig('figures/new/model2_bs32_lr002hc14k12_dmixed(57)_loss.pdf')
plt.show()

In [ ]:
a = [train[0] for train in  train_list]
b = [train[1] for train in train_list]

length = np.zeros(len(train_list))
for ix in range(len(train_list)):
    length[ix] = ix

fig, ax = plt.subplots()
ax.plot(length,a,label='Train')
ax.plot(length,b,label='Test')
ax.set(xlabel="Epochs",ylabel="Accuracy")
ax.set(title ='GraphConv')
ax.legend()
#plt.savefig('figures/new/model2_bs32_lr002hc14k12_dmixed(57).pdf')
plt.show()

In [ ]:
#PATH = "e250_bs32_lr0003_hc24k6_dmixed(579)_clean.pt"
torch.save(model.state_dict(),PATH)
#for param_tensor in model.state_dict():
#    print(param_tensor, "\t", model.state_dict()[param_tensor])
    

In [ ]:
#PATH = "e300_bs32_lr0003_hc24k6_largedataset.pt"
#torch.save(model.state_dict(),PATH)
#for param_tensor in model.state_dict():
#    print(param_tensor, "\t", model.state_dict()[param_tensor])
    
